In [146]:
import torch
import random

In [147]:
'''
    根据带有噪声的线性模型构造一个人造数据集
'''
def synthetic_data(w,b,num_example):
    X=torch.normal(0,1,(num_example,len(w)))
    y=torch.matmul(X,w)+b
    y+=torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

In [148]:
true_w=torch.tensor([2,-3.4])
true_b=4.2
features,labels=synthetic_data(true_w,true_b,1000)

In [149]:
'''
读取数据集，每次抽取一小批量样本
'''
def data_iter(batch_size,features,labels):
    num_examples=len(features)
    indices=list(range(num_examples)) # 生成索引
    random.shuffle(indices) # 随机打乱
    for i in range(0,num_examples,batch_size):
        batch_indices=torch.tensor(indices[i:min(i+batch_size,num_examples)])
        yield features[batch_indices],labels[batch_indices] # yield关键字定义生成器函数，，允许一次返回一个结果，而不是一次性返回所有的结果

In [150]:
# 初始化参数
w=torch.normal(0,0.01,size=(2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

In [151]:
'''
    定义模型
'''
def linreg(X,w,b):
    return torch.matmul(X,w)+b

In [152]:
'''
    定义损失函数
'''
def squared_loss(y_hat,y):
    return (y_hat-y.reshape(y_hat.shape))**2/2

In [153]:
'''
    定义优化算法
'''
def sgd(params,lr,batch_size):
    with torch.no_grad(): # 禁止梯度计算，防止更新过程中发生梯度计算
        for param in params:
            param-=lr*param.grad/batch_size # 更新参数
            param.grad.zero_() # 清空参数的梯度


In [154]:
# 准备
lr=0.03
num_epochs=3
net=linreg
loss=squared_loss
batch_size=10

In [155]:
# 训练
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l=loss(net(X,w,b),y)
        l.sum().backward() # 反向传播
        sgd([w,b],lr,batch_size)
    # 用更新的参数计算下损失，用于可视化
    with torch.no_grad():
        tran_l=loss(net(features,w,b),labels)
        print(f'epoch {epoch+1}, loss {float(tran_l.mean()):.4f}')


epoch 1, loss 0.0412
epoch 2, loss 0.0002
epoch 3, loss 0.0001


In [156]:
print(f'w的估计误差: {true_w-w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b-b}')

w的估计误差: tensor([ 0.0004, -0.0002], grad_fn=<SubBackward0>)
b的估计误差: tensor([4.1962e-05], grad_fn=<RsubBackward1>)
